In [4]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib
%pip install accelerate

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-1a_l6h_l
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-1a_l6h_l
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py) ... done
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached huggingface_hub-0.23.3-py3-none-any.whl (401 kB)
  Using cached beartype-0.18.5-py3-none-any.whl (917 kB)
  Using cached classifier_free_guidance_pytorch-0.6.4-py3-none-any.whl (1.4 MB)
  Using cached einops-0.8.0-py3-none-any.whl (43 kB)
  Using cached einx-0.3.0-py3-none-any.whl (102 kB)
  Using cached ema_pytorch-0.4.8-py3-none-any.whl (8.4 kB)
  Using cached local_attention-1.9.1-py3-none-any.whl (8.2 kB)
  Using cached gateloop_transformer-0.2.4-py3-none-any.whl (10 kB)
  Using cached pytorch_custom_utils-0.0.20-py3-none-any.whl (9.0 k

In [ ]:
from pathlib import Path 
import gc    
import torch
import os
import torch
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   

In [ ]:
%cd /root/mesh-transformer

In [ ]:
from accelerate import notebook_launcher
def training_function(num_processes=2):
    autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,   
        codebook_size = 2048, 
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8, 
        attn_decoder_depth  = 4,
        attn_encoder_depth = 2) 
    
    pkg = torch.load("./2k_autoencoder_490M_recon_0.3561_commit_-0.8175.pt")  
    autoencoder.load_state_dict(pkg['model'], strict = False) 
    
    dataset = MeshDataset.load("./objverse_250f_490.7M_all_17561_labels_568425_5_min_x5_aug.npz")  
    dataset2 = MeshDataset.load("./objverse_250f_98.1M_all_17561_labels_113685_5_min_x1_aug.npz")
    dataset.data.extend(dataset2.data)  
    dataset2 = MeshDataset.load("./shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")  
    dataset.data.extend(dataset2.data)  
    dataset2 = MeshDataset.load("./shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")  
    dataset.data.extend(dataset2.data) 
    dataset.sort_dataset_keys() 
         
    autoencoder.commit_loss_weight = 0.5
    autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                                 batch_size=32,
                                                 grad_accum_every =2,
                                                 learning_rate = 1e-3,
                                                 checkpoint_every_epoch=1)  
    loss1 = autoencoder_trainer.train(14445,  diplay_graph= False)        
 
args = ()
notebook_launcher(training_function, args, num_processes=4)